In [387]:
import numpy as np
import pywt
from scipy import signal
from sklearn.metrics import mean_squared_error

In [388]:
H_h = np.array([1, -1])/np.sqrt(2)
H_l = np.array([1, 1])/np.sqrt(2)

In [389]:
def down_sample(array ,sample_factor):
    array = np.array(array, dtype=float)
    array = array[::sample_factor]
    if (len(array.shape) == 2):
        array = array.transpose()
        array = array[::sample_factor]
    return array

In [390]:
from scipy.signal import convolve2d as conv2d
def convolve_2d(array_2d, kernel, mode):
    res = np.array(array_2d, dtype=float)
    kernel = np.array(kernel)
    if (mode == 'horizontal'):
        res = conv2d(res, kernel[None, :])
        res = res[:,len(kernel)-1:]
    if (mode == 'vertical'):
        res = conv2d(res, kernel[:, None])
        res = res[len(kernel)-1:,:]
    return res

In [391]:
def fried_model_gradient(image):
    tmp = convolve_2d(image, H_h, mode='horizontal')
    X = convolve_2d(tmp, H_l, mode='vertical')
    tmp = convolve_2d(image, H_l, mode='horizontal')
    Y = convolve_2d(tmp, H_h, mode='vertical')
    return X, Y

In [392]:
import random
exmpl = np.array([int(random.random() * 10) for i in range(16)])
exmpl = exmpl.reshape(4,4)
exmpl

array([[0, 4, 3, 5],
       [7, 4, 5, 9],
       [4, 7, 2, 2],
       [8, 2, 1, 7]])

In [393]:
coeffs = pywt.dwt2(exmpl, 'haar')
coeffs

(array([[  7.5,  11. ],
        [ 10.5,   6. ]]), (array([[-3.5, -3. ],
         [ 0.5, -2. ]]), array([[-0.5, -3. ],
         [ 1.5, -3. ]]), array([[-3.5,  1. ],
         [-4.5,  3. ]])))

In [394]:
coeffs2 = pywt.dwt2(coeffs[0], 'haar')
coeffs2

(array([[ 17.5]]), (array([[ 1.]]), array([[ 0.5]]), array([[-4.]])))

In [395]:
coeffs_hh = pywt.dwt2(coeffs[1][-1], 'haar')
coeffs_hh

(array([[-2.]]), (array([[-0.5]]), array([[-6.]]), array([[ 1.5]])))

In [396]:
X, Y = fried_model_gradient(exmpl)
[X, Y]

[array([[ 0.5,  0. ,  3. , -7. ],
        [ 0. , -2. ,  2. , -5.5],
        [-1.5, -3. ,  3. , -4.5],
        [-3. , -0.5,  3. , -3.5]]), array([[ 3.5,  1. ,  3. ,  2. ],
        [ 0. ,  0. , -5. , -3.5],
        [-0.5, -3. ,  2. ,  2.5],
        [-5. , -1.5, -4. , -3.5]])]

In [397]:
def process_next_X (prevX):
    tmp1 = convolve_2d(prevX, H_l, mode='horizontal')
    tmp2 = convolve_2d(tmp1, H_l, mode='horizontal')
    return down_sample(np.sqrt(2) * convolve_2d(tmp2, np.array([1,0,1]) / np.sqrt(2), mode='vertical'), 2)

In [398]:
def process_next_Y (prevY):
    tmp1 = convolve_2d(prevY, np.array([1,0,1]) / np.sqrt(2), mode = 'horizontal')
    tmp2 = convolve_2d(tmp1, H_l, mode = 'vertical')
    return np.sqrt(2) * down_sample(convolve_2d(tmp2, H_l, mode = 'vertical'), 2)

In [399]:
def process_next_HH (X):
    tmp1 = convolve_2d(X, H_l, mode = 'horizontal')
    tmp2 = convolve_2d(tmp1, H_l, mode = 'horizontal')
    tmp3 = convolve_2d(tmp2, np.array([1,0,-1]) / np.sqrt(2), mode = 'vertical')
    return np.sqrt(2) * down_sample(tmp3, 4)

In [400]:
def process_left_quadrant (grad_X, grad_Y, M):
    X = dict({M : grad_X})
    Y = dict({M : grad_Y})
    HL = dict({M - 1 : -down_sample(X[M], 2).T})
    LH = dict({M - 1 : -down_sample(Y[M], 2).T})
    HH = dict()
    for k in range(2, M + 1)[::-1]:
        X[k - 1] = process_next_X(X[k])
        Y[k - 1] = process_next_Y(Y[k])
        LH[k - 2] = -down_sample(Y[k - 1], 2).T
        HL[k - 2] = -down_sample(X[k - 1], 2).T
        HH[k - 2] = process_next_HH(X[k])
    return LH, HL, HH

In [401]:
def process_next_X_right(prevX, k, M):
    if (k == M):
        tmp1 = convolve_2d(X, np.array([1, 0, 1]) / np.sqrt(2), mode = 'horizontal')
        tmp2 = convolve_2d(tmp1, H_h, mode = 'vertical')
        tmp3 = convolve_2d(tmp2, H_h, mode = 'vertical')
        return np.sqrt(2) * down_sample(tmp3, 2)
    else:
        tmp1 = convolve_2d(X, np.array([1, 0, 1]) / np.sqrt(2), mode = 'horizontal')
        tmp2 = convolve_2d(tmp1, H_l, mode = 'vertical')
        tmp3 = convolve_2d(tmp2, H_l, mode = 'vertical')
        return np.sqrt(2) * down_sample(tmp3, 2)

In [402]:
def process_next_Y_right(prevY, k, M):
    if (k == M):
        tmp1 = convolve_2d(prevY, H_h, mode = 'horizontal')
        tmp2 = convolve_2d(tmp1, H_h, mode = 'horizontal')
        tmp3 = convolve_2d(tmp2, np.array([1, 0, 1]) / np.sqrt(2), mode = 'vertical')
        return np.sqrt(2) * down_sample(tmp3, 2)
    else:
        tmp1 = convolve_2d(prevY, H_l, mode = 'horizontal')
        tmp2 = convolve_2d(tmp1, H_l, mode = 'horizontal')
        tmp3 = convolve_2d(tmp2, np.array([1, 0, 1]) / np.sqrt(2), mode = 'vertical')
        return np.sqrt(2) * down_sample(tmp3, 2)

In [403]:
def process_next_HH_right(X, k, M):
    if (k == M):
        tmp1 = convolve_2d(X, np.array([1, 0, -1]) / np.sqrt(2), mode = 'horizontal')
        tmp2 = convolve_2d(tmp1, H_h, mode = 'vertical')
        tmp3 = convolve_2d(tmp2, H_h, mode = 'vertical')
        return np.sqrt(2) * down_sample(tmp3, 4)
    else:
        tmp1 = convolve_2d(X, np.array([1, 0, -1]) / np.sqrt(2), mode = 'horizontal')
        tmp2 = convolve_2d(tmp1, H_l, mode = 'vertical')
        tmp3 = convolve_2d(tmp2, H_l, mode = 'vertical')
        return np.sqrt(2) * down_sample(tmp3, 4)

In [404]:
def process_right_quardrant (grad_X, grad_Y, M):
    X = dict({M : grad_X})
    Y = dict({M : grad_Y})
    HL = dict()
    LH = dict()
    HH = dict()
    for k in range(2, M + 1)[::-1]:
        X[k - 1] = process_next_X_right(X[k], k, M)
        Y[k - 1] = process_next_Y_right(Y[k], k, M)
        LH[k - 2] = -down_sample(X[k - 1], 2)
        HL[k - 2] = -down_sample(Y[k - 1], 2)
        HH[k - 2] = process_next_HH_right(X[k], k, M)
    return LH, HL, HH

In [405]:
def synthesis(LL, LH, HL, HH, M):
    for k in range(M - 1):
        LL = pywt.idwt2([LL, (LH[k], HL[k], HH[k])], 'haar')
    return LL

In [406]:
def get_image_from_gradient(grad_X, grad_Y, mean = 1, ll_right = np.array([[0.]])):
    M = int(np.log2(len(grad_X)))
    LL_0_left = np.array([[(2 ** M) * mean]])
    LH_left, HL_left, HH_left = process_left_quadrant (grad_X, grad_Y, M)
    LL = synthesis(LL_0_left, LH_left, HL_left, HH_left, M)
    LL_0_right = ll_right
    LH_right, HL_right, HH_right = process_right_quardrant(grad_X, grad_Y, M)
    HH = synthesis(LL_0_right, LH_right, HL_right, HH_right, M)
    HL = down_sample(grad_X, 2)
    LH = down_sample(grad_Y, 2)
    return pywt.idwt2([LL, (LH, HL, HH)], 'haar')

In [407]:
res = get_image_from_gradient(X, Y, np.mean(exmpl), np.array([[-1.5]]))
res

array([[ 4.125,  6.875,  5.125,  5.375],
       [ 3.875,  0.125,  4.375,  7.125],
       [ 6.125,  7.375,  7.125,  0.875],
       [ 7.375,  0.125,  1.875,  2.125]])

In [408]:
mean_squared_error(res, exmpl)

7.453125